In [1]:
!pip install pymilvus ollama llama-index-llms-ollama llama-index-vector-stores-milvus

The following packages are already present in the pyproject.toml and will be skipped:

  - pymilvus
  - ollama
  - llama-index-llms-ollama
  - llama-index-vector-stores-milvus

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [1]:
!pip install llama-index-embeddings-jinaai llama-index-readers-file

The following packages are already present in the pyproject.toml and will be skipped:

  - llama-index-embeddings-jinaai
  - llama-index-readers-file

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [9]:
!pip install sentence-transformers llama-index-embeddings-huggingface

Using version ^2.7.0 for sentence-transformers
Using version ^0.2.0 for llama-index-embeddings-huggingface

Updating dependencies
Resolving dependencies... (0.9s)Resolving dependencies... (0.4s)

Package operations: 2 installs, 0 updates, 0 removals

  - Installing minijinja (2.0.1): Pending...
  - Installing minijinja (2.0.1): Downloading... 0%
  - Installing minijinja (2.0.1): Downloading... 3%
  - Installing minijinja (2.0.1): Downloading... 7%
  - Installing minijinja (2.0.1): Downloading... 10%
  - Installing minijinja (2.0.1): Downloading... 13%
  - Installing minijinja (2.0.1): Downloading... 17%
  - Installing minijinja (2.0.1): Downloading... 20%
  - Installing minijinja (2.0.1): Downloading... 22%
  - Installing minijinja (2.0.1): Downloading... 28%
  - Installing minijinja (2.0.1): Downloading... 30%
  - Installing minijinja (2.0.1): Downloading... 38%
  - Installing minijinja (2.0.1): Downloading... 40%
  - Installing minijinja (2.0.1): Downloading... 47%
  - Installing min

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_local_rag.db")

In [ ]:
client.list_collections()

In [14]:
import os

from dotenv import load_dotenv

load_dotenv()
JINA_AI_API_KEY = os.getenv("JINA_AI_API_KEY")

In [15]:
from llama_index.embeddings.jinaai import JinaEmbedding

#JinaAI Embeddings
jina_embedding_model = JinaEmbedding(
    api_key=JINA_AI_API_KEY,
    model="jina-embeddings-v2-base-de",
)

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from sentence_transformers import SentenceTransformer

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


# embeddings = model.encode([
#     'How is the weather today?',
#     'Wie ist das Wetter heute?'
# ])
# print(cos_sim(embeddings[0], embeddings[1]))


KeyboardInterrupt: 

In [4]:
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core import StorageContext, ServiceContext

llm = Ollama(model="llama3", request_timeout=120.0)

service_context_jina = ServiceContext.from_defaults(llm=llm, embed_model=model, chunk_size=350)

vector_store_jina = MilvusVectorStore(
    uri="milvus_local_rag.db",
    collection_name="st_embeddings",
    dim=384,  # the value changes with embedding model
    overwrite=False  # drop table if exist and then create
    
    )
storage_context_jina = StorageContext.from_defaults(vector_store=vector_store_jina)

/var/folders/kv/3dw9p_ts4b114chqt9m027pc0000gn/T/ipykernel_65311/808032196.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_jina = ServiceContext.from_defaults(llm=llm, embed_model=model, chunk_size=350)


In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [6]:
docs = SimpleDirectoryReader(input_dir="data/pdfs/", num_files_limit=1000).load_data(num_workers=8)

In [7]:
vector_index_jina = VectorStoreIndex.from_documents(docs, storage_context=storage_context_jina, service_context=service_context_jina)

AttributeError: 'SentenceTransformer' object has no attribute 'get_text_embedding_batch'

In [26]:
from llama_index.core.tools import RetrieverTool, ToolMetadata

milvus_tool_openai = RetrieverTool(
    retriever=vector_index_jina.as_retriever(similarity_top_k=3),  # retrieve top_k results
    metadata=ToolMetadata(
        name="CustomRetriever",
        description='Retrieve relevant information from provided documents.'
    ),
)

In [34]:
query_engine = vector_index_jina.as_query_engine()
response = query_engine.query("How many services use a Fax Machines in Berlin?")

In [35]:
response.response

'According to the provided context, it is mentioned that the query "Über wie viele Faxgeräte verfügt die Berliner Verwaltung insgesamt?" (How many fax machines does the Berlin administration have in total?) has an answer of 5,333.'